In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from cs_bahtml.gospa import calculate_gospa_distance

In [3]:
df = pd.read_json('../data/parsed_util.json')

In [4]:
df

,map_name,entity_id,round,throwSeconds,grenade_type,throwerName,throwerTeam,throwerBuyType,opponentTeam,opponentBuyType,throwerSide,throwCords,landCords,starting_positions,dead_players,alive_players
0,de_ancient,65,7,19.984375,molotov,jabbi,Astralis,force,FaZe Clan,full,TERRORIST,"[-1545.9470214844, 590.7413330078, 73.03125]","[-1448.5625, 118.78125, 94.15625]","[[-950.1359863281, -755.3543701172, 91.3916091...","[[-1266.0162353516, 670.618347168, 59.51692199...","[[-1944.0313720703, 465.9612731934, 75.03125]]"
1,de_ancient,65,11,0.515625,smoke,br0,Astralis,full,FaZe Clan,full,TERRORIST,"[-392.0, -2224.0, -117.9411621094]","[-808.5625, -664.84375, 41.84375]","[[-409.4080810547, -2295.2575683594, -163.2557...",None,"[[-1341.1284179688, -1152.1937255859, -12.2715..."
2,de_ancient,65,12,8.578125,smoke,Staehr,Astralis,full,FaZe Clan,full,TERRORIST,"[-1182.9777832031, -936.0553588867, 34.6235275...","[-1134.09375, -870.0, 32.65625]","[[-1182.9777832031, -936.0553588867, 34.623527...",None,"[[-775.7543945312, -879.1306762695, 38.7095680..."
3,de_ancient,71,7,8.046875,molotov,Staehr,Astralis,force,FaZe Clan,full,TERRORIST,"[-1135.5604248047, -880.2910766602, 34.5519714...","[-382.5, -524.5625, 365.0]","[[-1135.5604248047, -880.2910766602, 34.551971...","[[-1266.0162353516, 670.618347168, 59.51692199...","[[-1437.0113525391, -1099.2087402344, -6.02288..."
4,de_ancient,74,11,11.484375,molotov,karrigan,FaZe Clan,full,Astralis,full,CT,"[1201.6058349609, 606.6052246094, 125.6050643921]","[1152.25, -728.75, 15.375]","[[-253.7420654297, -192.2243652344, 68.0311584...",None,"[[51.9366378784, -679.7330322266, 158.81071472..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,de_vertigo,1012,2,8.093750,smoke,Staehr,Astralis,full,Virtus.Pro,full,TERRORIST,"[-906.1011962891, -121.7451782227, 11488.03125]","[-681.90625, -942.71875, 11630.21875]","[[-1273.4239501953, -661.0948486328, 11776.582...",None,"[[-1793.4641113281, -534.6804199219, 11552.535..."
1093,de_vertigo,1014,2,9.343750,molotov,Staehr,Astralis,full,Virtus.Pro,full,TERRORIST,"[-814.6753540039, -333.8669433594, 11488.03125]","[-1076.15625, -1122.3125, 11778.1875]","[[-1423.9686279297, -824.96875, 11776.03125], ...",None,"[[-1897.8319091797, -421.1993408203, 11552.031..."
1094,de_vertigo,1016,16,12.593750,molotov,Jame,Virtus.Pro,full,Astralis,full,TERRORIST,"[-1423.96875, -824.96875, 11776.03125]","[-1057.75, -723.875, 12018.03125]","[[-818.6741333008, -507.5389404297, 11522.9169...","[[-2416.3474121094, -81.0317382812, 11712.0312...","[[-913.0172119141, -549.021484375, 11592.03125..."
1095,de_vertigo,1017,8,28.015625,smoke,device,Astralis,full,Virtus.Pro,full,TERRORIST,"[-1807.3677978516, 545.7968139648, 11776.03125]","[-1523.59375, 778.78125, 11778.03125]","[[-1807.3677978516, 545.7968139648, 11776.0312...","[[-692.0317993164, -473.9997558594, 11514.5322...","[[-1394.15625, 753.0313110352, 11776.03125], [..."


In [67]:
from scipy.spatial.distance import pdist, squareform, cdist
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import numpy as np
# Define a function to calculate Euclidean distance
def calculate_distance(cords):
    return pdist(cords, 'euclidean')
def calculate_equal_distance(series):
    x = np.asarray(series)
    return squareform((x[:, None] != x[None, :]).astype(int))*1e9

def calculate_clusters(df, threshold=200):
    # Create distance matrices for throwCords and landCords
    throwCords = df['throwCords'].tolist()
    landCords = df['landCords'].tolist()
    throwCords_dist = calculate_distance(throwCords)
    landCords_dist = calculate_distance(landCords)

    map_name_dist = calculate_equal_distance(df['map_name'])
    throwerSide_dist = calculate_equal_distance(df['throwerSide'])
    grenade_type_dist = calculate_equal_distance(df['grenade_type'])
    max_grouping_dist = np.maximum.reduce([map_name_dist, throwerSide_dist, grenade_type_dist])

    
    # Use the maximum distance between throwCords and landCords
    max_dist = np.maximum.reduce([throwCords_dist, landCords_dist, max_grouping_dist])

    # Use this distance matrix in hierarchical clustering
    clusters = linkage(squareform(max_dist), method='complete')
    clusters = fcluster(clusters, threshold, criterion='distance')
    return clusters


In [74]:

df['cluster'] = calculate_clusters(df, threshold = 500)
clusters_sorted = df.cluster.value_counts().index.tolist()

C:\Users\Baht\AppData\Local\Temp\ipykernel_8112\3319649675.py:28: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  clusters = linkage(squareform(max_dist), method='complete')


In [72]:
from cs_bahtml.plotting import add_smoke_wall, add_players
from cs_bahtml.radars import RADARS
def plot_cluster(df, cluster):
    plot_df = df[df.cluster == cluster]
    map_name = plot_df.map_name.iloc[0]
    assert plot_df.map_name.nunique() == 1
    fig = RADARS[map_name].copy()
    for idx, row in plot_df.iterrows():
        fig = add_smoke_wall(fig, [row['landCords']], map_name, type = row['grenade_type'])
        fig = add_players(fig, [row['throwCords']], map_name, type = 'thrower')
    fig.show()

In [80]:

map_df = df.query("map_name == 'de_nuke'")
clusters_sorted = map_df.cluster.value_counts().index.tolist()

for cluster in clusters_sorted[:10]: 
    print(cluster)
    
    plot_cluster(map_df, cluster)

189
316
176
229
235
222
215
252
216
231


In [103]:
# from scipy.spatial.distance import euclidean
# import numpy as np
# def calculate_score(df, max_dist = 200, min_occurences = 2):
#     df = df.merge(df, on=['map_name', 'throwerTeam','throwerSide','grenade_type'], suffixes=('', '_other'), how='left')
#     df['throw_distance'] = df.apply(lambda row: euclidean(row.throwCords, row.throwCords_other), axis=1)
#     df['land_distance'] = df.apply(lambda row: euclidean(row.landCords, row.landCords_other), axis=1)
#     df = df.query('throw_distance < @max_dist and land_distance < @max_dist')
#     df = df.groupby(['map_name','entity_id','round', 'throwerTeam','throwerSide','grenade_type']).filter(lambda x: len(x) > min_occurences)
#     player_df = df.groupby(['map_name','entity_id','round', 'throwerTeam','throwerSide','grenade_type'])['alive_players_other'].agg(list).reset_index(name='players')
#     entities_df = df.groupby(['map_name','entity_id','round', 'throwerTeam','throwerSide','grenade_type'])['entity_id_other'].agg(set).reset_index(name='entity_id_set')
#     player_df['util_score'] = player_df.apply(lambda row: calculate_gospa_distance(row['players'],c = 2000, p = 1, alpha = 2, map=row['map_name'], euclidean_dist=True), axis = 1)

#     player_df = player_df[['map_name','entity_id','round', 'throwerTeam','throwerSide','grenade_type','util_score']]
#     df = df.merge(player_df, on=['map_name','entity_id','round', 'throwerTeam','throwerSide','grenade_type'], how='left')
#     df = df.merge(entities_df, on=['map_name','entity_id','round', 'throwerTeam','throwerSide','grenade_type'], how='left')
#     df['players'] = df['alive_players']
#     df = df.drop_duplicates(subset=['map_name','entity_id','round', 'throwerTeam','throwerSide','grenade_type'])
    
#     return df[['map_name','entity_id','round','throwSeconds', "throwCords", "landCords",'throwerName', 'throwerTeam','opponentTeam','throwerSide','grenade_type','util_score', 'starting_positions','players','dead_players', 'entity_id_set']]


In [82]:
df.to_json('../data/parsed_clusters.json')